# VISUALIZE OUTPUTS FROM ANALYSIS FUNCTIONS

In [ ]:
# init
import importlib
import tTsTGrpUtils as tsutil
import utils_plots as uplots
import os
import sys
sys.path.append("/host/verges/tank/data/daniel/01_3T7T/z/code/analyses/hippunfold_toolbox/")
from hippunfold_toolbox import plotting
import numpy as np
import pandas as pd
from brainspace.datasets import load_mask

test = False # for quick testing

# Data paths
root = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/"
pth_05b_winComp_ic = "05b_stats_winStudy_grp_30Nov2025-114034.pkl"
pth_05c_winD = "05c_stats_winD_30Nov2025-114813.pkl" # Has all data from above maps as well. D score btw px and ctrl z-scores
pth_05d_btwD = "05d_btwD_30Nov2025-125409.pkl" # difference in D scores between studies

#dl_winD = tsutil.loadPickle(root + pth_05c_winD)
dl_winD = tsutil.loadPickle(root + pth_05c_winD)
dl_btwD = tsutil.loadPickle(root + pth_05d_btwD)

# Specifications for dl filtering
key_ft = 'feature' # key for feature in dict
foi = ['thickness', 'T1map'] # features of interest
key_ids = [['ctrl_IDs'],['TLE_R_IDs', 'TLE_L_IDs']]


In [ ]:
# Participant wise Z-scoring figures (control distributions in black)
importlib.reload(uplots)

reimport_src = False

if 'dl_winComp_ic' not in globals() or reimport_src:
    dl_winComp_ic = tsutil.loadPickle(root + pth_05b_winComp_ic, verbose = True)

dl_winComp_ic = tsutil.filt_dl(dl = dl_winComp_ic, 
                                   key = key_ft, voi = foi, 
                                   key_ids = key_ids)

# NOTE. pass z-scores only. Need changes for w-scores
uplots.raw2Z_vis(dl = dl_winComp_ic, save_path = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/rawToZ", 
               key_dfs_raw = ['df_maps_parc-glsr_mean', 'df_maps_parc-dk25_mean'],
               marks = True)

In [ ]:
# Z-score distributions for pt and controls with corresponding D-scores
importlib.reload(uplots)

save_path = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/zToD" # will save raw images to ./raw
reimport_src = False
test = False
key_df_d = 'df_maps_parc-glsr_mean_z_d_ic'
idx_d = 'd_TLE_ic_ipsiTo-L' # index of cohen's d values in df_d

if 'dl_winD' not in globals() or reimport_src:
    dl_winD = tsutil.loadPickle(root + pth_05c_winD, verbose = True)

dl_winD = tsutil.filt_dl(dl = dl_winD, 
                        key_ft = key_ft, foi = foi, 
                        key_ids = key_ids)

uplots.z2D_vis(dl=dl_winD, key_df_d = key_df_d, save_path = save_path,
                idx_d = idx_d, test = test)

In [ ]:
# Comparison between 3T and 7T Cohen's Ds  
importlib.reload(uplots)

test = False
reimport_src = True

if 'dl_btwD' not in globals or reimport_src:
       dl_btwD = tsutil.loadPickle(root + pth_05d_btwD, verbose = True)

dl_btwD = tsutil.filt_dl(dl = dl_btwD, 
                        key_ft = key_ft, foi = foi, 
                        key_ids = key_ids)

# TODO. Plot these metrics on cortical, hippocampal surfaces; break down by vertical bar plot;
uplots.btwD_vis(dl = dl_btwD, save_path = save_path,
              key_winD = 'df_d_ic', idx_winD = ['d_TLE_ic_ipsiTo-L'], 
              key_btwD = 'comps_df_d_ic', idx_btwD = ['d_TLE_ic_ipsiTo-L_Δd', 'd_TLE_ic_ipsiTo-L_Δd_by3T', 'd_TLE_ic_ipsiTo-L_Δd_by7T', 'd_TLE_ic_ipsiTo-L_Δd_bySum', 'd_TLE_ic_ipsiTo-L_Δd_byMean'], 
              test = test)


In [ ]:
# % of vertices within each parcel with extreme z-values
importlib.reload(tsutil)
importlib.reload(uplots)

reimport_src = False
zThresh = 3 # z-threshold

# TODO. Add functionality to make this a % of vertices within parcel
thresh_nVrtx = 10 # min number of vertices exceeding threshold to count

if reimport_src:
    dl_winD = tsutil.loadPickle(root + pth_05c_winD, verbose = True)
    dl_winD = tsutil.filt_dl(dl = dl_winD, 
                            key_ft = key_ft, foi = foi, 
                            key_ids = key_ids)

# A. Compute extreme vertex counts per parcel and save to dfs in each item in dl
dl_vrtxCounts, path = tsutil.get_xtremeVrtx(dl = dl_winD, key_df = 'df_maps_z', thresh = zThresh, thresh_nVrtx = thresh_nVrtx,
                                     lobes = True,
                                     save_path_dfs = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/dfs/06a_xtremeVrtxPerParc",
                                     save_path_dl_out = "/host/verges/tank/data/daniel/01_3T7T/z/outputs",
                                     save_name = f'06a_xtremeVrtxPerParc',
                                     test = False)


In [ ]:

print(type(dl_vrtxCounts))
print(len(dl_vrtxCounts))
print(dl_vrtxCounts[0].keys())
df = tsutil.loadPickle(dl_vrtxCounts[0]['xtremeVrtxPerParc_df_maps_z_thresh-2_ctrl_lobes_stats'])
print(df.index)


In [ ]:
importlib.reload(uplots)
# B. Visualize with horizontal bar graphs
count = "pt"
save_path = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/06a_xtrmeVrtxPerParc/raw"
pdf_svPth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/06a_xtrmeVrtxPerParc"
pdf_svPth = None

if count == "vrtx":
    nums = ["mean"]
    denom = "v_vertices"
elif count == "pt":
    nums = ["n_pts_withXtremeVrtx"]
    denom = "n_pts"
grps = ["TLE_ic", "ctrl"]
test = False

# TODO. For hippocampal segmentations, replace index with interpretable name
print(f"Saving plots to: {save_path}")
for num in nums:
    for grp in grps:
        print(f"Visualizing {num} extreme vertices per parcel/region for group: {grp}...")
        key_df_ctx = f'xtremeVrtxPerParc_df_maps_z_thresh-{zThresh}_{grp}_lobes_stats'
        key_df_hip = f'xtremeVrtxPerParc_df_maps_z_thresh-{zThresh}_{grp}_stats'

        uplots.percentXtremeVrtxPerParc_vis(dl = dl_vrtxCounts, grp = grp, key_df_ctx = key_df_ctx, key_df_hip = key_df_hip, numerator = num, denominator = denom, 
                                            save_path = save_path, test = test)
    
    if pdf_svPth:
        uplots.pdf_percXtremeVrtx(dl = dl_vrtxCounts, nums = [num], key_df_ctx = key_df_ctx, key_df_hip = key_df_hip, 
                           fig_dir = save_path, pdf_svPth = pdf_svPth, cleanup = False, verbose= True)


In [ ]:
importlib.reload(tsutil)
#tsutil.print_dict(dl_btwD)
dl_btwD_filt = tsutil.filt_dl(dl = dl_btwD, 
                        key = 'surf', voi = "fsLR-5k")
tsutil.print_dict(dl_winD)

In [ ]:
df_name = "df_maps_z"
item = dl_winD[6]
df = tsutil.loadPickle(item[df_name])

df.index

In [ ]:
# Plot TLE mean, CTRL mean, CTRL std, Cohen's D maps on brain surfaces

importlib.reload(tsutil)
importlib.reload(uplots)
import pandas as pd


indices = [4,5]
save_pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/surfaces/fig1/thickness_32k/raw"
key = 'df_maps'

d_key = 'df_maps_z_d_ic'
d_metric = 'd_TLE_ic_ipsiTo-L'

for index in indices:
    
    item = dl_winD[index]
    tsutil.printItemMetadata(item, idx = index)

    study = item['study']
    region = item['region']
    surf = item['surf']
    ft = item['feature']
    lbl = item['label']
    res = item['downsampledRes']
    smth = item['smth']
    
    df = tsutil.loadPickle(item[key])
    
    # apply midsurface mask
    if region in ["cortex", "ctx"]:
        lh_orig, rh_orig = tsutil.splitHemis(df)
        lh_msk, rh_msk = tsutil.apply_midMask(lh_orig, rh_orig, surf = surf)
        df = pd.concat([lh_msk, rh_msk], axis=1)
    
    ids_l = item['TLE_L_IDs'] + item['ctrl_IDs']
    ids_r = item['TLE_R_IDs']
    #print(f"TLE: {len(item['TLE_L_IDs']) + len(item['TLE_R_IDs'])} | Ctrl: {len(item['ctrl_IDs'])}")
    #print(f"\tTLEs: {item['TLE_L_IDs'] + item['TLE_R_IDs']}")
    
    df_l = df.loc[ids_l]
    df_r = df.loc[ids_r]

    df_ic = tsutil.toIC(df_r = df_r, df_l = df_l)
    df_ctrl_ic = df_ic.loc[item['ctrl_IDs']]
    df_tle_ic = df_ic.loc[item['TLE_L_IDs'] + item['TLE_R_IDs']]
    
    save_name_base = f"{index}-{study}_{region}_{surf}_{ft}_{lbl}_smth-{smth}mm_res-{res}_key-{key}"
    
    if study in ["7T", "PNI"]:
        cbar_pos = 'left'
    else:
        cbar_pos = 'right'
    
    for grp in ['ctrl_ic', 'tle_ic']:
        if grp == 'ctrl_ic':
            df_grp = df_ctrl_ic
        else:
            df_grp = df_tle_ic

        mean_map = df_grp.mean()
        lh, rh = tsutil.splitHemis(mean_map)

        std_map = df_grp.std()
        std_lh, std_rh = tsutil.splitHemis(std_map)
        print(f"\tMean shapes (l,r): {lh.shape} {rh.shape}")
        print(f"\tSTD shapes (l,r): {std_lh.shape} {std_rh.shape}")
        
        save_name_grp = f"{save_name_base}_grp-{grp}"
        """
        min_mn, max_mn, cmap_mn = uplots.get_maxVals(mean_map, ft, metric = 'mean', region = region, verbose =False)
        uplots.showBrain(mean_lh, mean_rh, 
                         region = region, surface = surf,
                         feature_lbl = f"{ft} (Mean)", ipsiTo = 'L',
                         title = f"Mean {grp} - {item['feature']} - {item['surf']}",
                         min_val = min_mn, max_val = max_mn, cmap = cmap_mn,
                         cbar_pos = cbar_pos,
                         save_name = f"{save_name_grp}_mean",
                         save_pth = save_pth)
        """
        min_std, max_std, cmap_std = uplots.get_maxVals(std_map, ft, metric = 'std', region = region, verbose =False)
        uplots.showBrain(std_lh, std_rh, 
                         surface = surf, region = region, 
                         feature_lbl = f"{ft} (StD)", ipsiTo = 'L',
                         title = f"STD {grp} - {item['feature']} - {item['surf']}",
                         min_val = min_std, max_val = 0.3, cmap = cmap_std,
                         cbar_pos = cbar_pos,
                         save_name = f"{save_name_grp}_std",
                         save_pth = save_pth)
    
    # Cohen's D map:
    df_d = tsutil.loadPickle(item[d_key])
    df_d = df_d.loc[d_metric]
    lh_d, rh_d = tsutil.splitHemis(df_d)
    if region in ["cortex", "ctx"]:
        lh_d, rh_d = tsutil.apply_midMask(lh_d, rh_d, surf = surf)

    print(f"\tCohen's D shapes (l, r): {lh_d.shape} {rh_d.shape}")
    min_d, max_d, cmap_d = uplots.get_maxVals(lh_d, ft, metric = d_metric, region = region, verbose =False)
    uplots.showBrain(lh_d, rh_d, 
                    surface = surf, region = region, 
                    feature_lbl = f"{ft} (Cohen's D)", ipsiTo = 'L',
                    title = f"Cohen's D {grp} - {item['feature']} - {item['surf']}",
                    min_val = min_d, max_val = max_d, cmap = cmap_d,
                    cbar_pos = cbar_pos,
                    save_name = f"{save_name_base}_D",
                    save_pth = save_pth)

In [ ]:
# cohen's D difference maps (different metrics)
tsutil.print_dict(dl_btwD)

In [ ]:
# Plot dDifference metrics
importlib.reload(tsutil)
importlib.reload(uplots)

# 10mm ctx
# 2mm hip
# 32k, midthickness
# T1map, thickness

indices = [2,10,19, 27, 30, 38] # to determine
save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/dD"
key = 'comps_df_maps_z_d_ic'
metrics = ['d_TLE_ic_ipsiTo-L_Δd', 
           'd_TLE_ic_ipsiTo-L_Δd_by3T', 'd_TLE_ic_ipsiTo-L_Δd_by7T', 
           'd_TLE_ic_ipsiTo-L_Δd_bySum', 'd_TLE_ic_ipsiTo-L_Δd_byMean']
cbar_pos = 'right'

for index in indices:
    
    item = dl_btwD[index]
    tsutil.printItemMetadata(item, idx = index)

    studies = item['studies']
    region = item['region']
    surf = item['surf']
    ft = item['feature']
    lbl = item['label']
    res = item['downsampledRes']
    smth = item['smth']
    save_name_base = f"dD_{region}-{ft}_{surf}_{lbl}_smth-{smth}mm_res-{res[0]}-{res[1]}_key-{key}"
    
    df = tsutil.loadPickle(item[key])
    
    # apply midsurface mask
    if region in ["cortex", "ctx"]:
        lh_orig, rh_orig = tsutil.splitHemis(df)
        lh_msk, rh_msk = tsutil.apply_midMask(lh_orig, rh_orig, surf = surf)
        df = pd.concat([lh_msk, rh_msk], axis=1)
    
    # Check if data is already in ipsi/contra format
    sample_column = df.columns[0]
    needs_ic_conversion = '_ipsi' not in sample_column and '_contra' not in sample_column

    if needs_ic_conversion:
        df_l = df.loc[ids_l]
        df_r = df.loc[ids_r]
        df_ic = tsutil.toIC(df_r = df_r, df_l = df_l)
    else:
        df_ic = df

    for met in metrics:
        metric = df_ic.loc[met]
        
        
        lh, rh = tsutil.splitHemis(metric)

        print(f"\tMetric `{met}` shapes (l,r): {lh.shape} {rh.shape}")
        
        save_name_met = f"{save_name_base}_metric-{met}"
        cmap = 'seismic'
        min_val = -1.5
        max_val = 1.5
        
        if '_by' in met:
            min_val = -5
            max_val = 5
        
        uplots.showBrain(lh, rh, 
                         surface = surf, region = region, 
                         feature_lbl = f"{ft} ({met})", ipsiTo = 'L',
                         title = f"dD-{met} - {ft} {surf} {lbl} {smth}mm 3T-{res[0]}, 7T-{res[1]}",
                         min_val = min_val, max_val = max_val, cmap = cmap,
                         cbar_pos = cbar_pos,
                         save_name = save_name_met,
                         save_pth = save_pth)


In [ ]:
#tsutil.print_dict(dl_btwD)
idx = 10
item = dl_btwD[idx]
tsutil.printItemMetadata(item)
print(item.keys())

df = tsutil.loadPickle(item[f'comps_df_maps_z_d_ic'])
print(f"df <{df.shape}>:\n\tIndex: {df.index.tolist()}\n\tColumns (first 5): {df.columns.tolist()[:5]}...{df.columns.tolist()[-5:]}")

nan_counts = df.isna().sum()
nan_counts = nan_counts[nan_counts > 0]
print(f"NaNs: {nan_counts if nan_counts.shape[0]>1 else 'no NaNs'}")
'comps_df_maps_z_d_ic'
metrcs = ['d_TLE_ic_ipsiTo-L_Δd', 'd_TLE_ic_ipsiTo-L_Δd_by3T', 'd_TLE_ic_ipsiTo-L_Δd_by7T', 'd_TLE_ic_ipsiTo-L_Δd_bySum', 'd_TLE_ic_ipsiTo-L_Δd_byMean']

In [ ]:
# create bar graph by lobe/parcel
# one bar per metric


In [ ]:
parc = item['parcellation']
if parc == 'glasser':
    parc = 'glsr'
else:
    parc = parc.lower()
print(parc)


print(f"{len(df.columns.tolist())}: {df.columns.tolist()} ")

In [ ]:
importlib.reload(uplots)

items = [item]
print(type(items))
df_names = 'comps_df_maps_z_d_ic'
metrics = ['m_TLE_ic_ipsiTo-L_3T', 'm_ctrl_3T', 'std_ctrl_3T', 'd_TLE_ic_ipsiTo-L_3T']
cbar_lbl = ["MEAN [TLE]", "MEAN [CTRL]", "St Dev [CTRL]", "Cohen's D"]
title = tsutil.printItemMetadata(item, printStudy = True, return_txt=True)
uplots.plotNsurfaces(items=items, df_names = df_names, metrics = metrics, lbls = cbar_lbl, title = title)